In [ ]:
import pickle

In [2]:
with open('pickles/midi_df.pickle', 'rb') as f:
    songs = pickle.load(f)

In [3]:
songs.head()

,file_name,seconds_length,instruments,avg_notes
0,FridayThe13th_-_MapDark.mid,93.272727,"[Pipe Organ, Electric Organ, Harpsichord, Stri...",468.000000
1,cv1-4.mid,41.533333,"[Electric Organ, Bass, Piano, Violoncello]",149.250000
2,MM3-Snake_Man.mid,124.125874,"[BASS (FINGER), Electric Bass, SYNTH BASS, Sam...",606.250000
3,dw4batl.mid,115.231788,"[Electric Guitar, Bass, Electric Bass, Bass Su...",340.400000
4,Balloon_Fight_-_Main_Theme_%28Dancing_Balloon%...,11559.700000,"[Steel Drum, Bass, Sampler, Synth bass, None, ...",414.571429


In [4]:
songs = songs.dropna()

In [5]:
with_sampler = songs['instruments'].apply(lambda x: 'Sampler' in x)

In [6]:
songs[with_sampler].describe()

,seconds_length,avg_notes
count,2229.000000,2229.000000
mean,101.157816,414.378286
std,337.068819,502.182123
min,0.000000,0.636364
25%,47.272727,164.250000
50%,79.583333,309.000000
75%,114.352941,523.750000
max,11559.700000,11872.000000


In [7]:
average_seconds = songs['seconds_length'].between(50, 80)

In [8]:
average_notes = songs['avg_notes'].between(165, 310)

In [9]:
most_average_songs = songs[with_sampler & average_seconds & average_notes]

In [10]:
sample = most_average_songs.sample(25)

In [11]:
random_sample = list(sample['file_name'])

In [12]:
random_sample

['UlimatepasswordmetalgearNES.mid',
 'RM4-TitleTheme-X.mid',
 'BubblemanByCryogen.mid',
 'ff1temp2.mid',
 'SMB3_-_Grass_Land.mid',
 'StarTropics-miracola.mid',
 'Master6.mid',
 'Bombman1.mid',
 'mario3-world4.mid',
 'lemtensp.mid',
 'Klxfairy.mid',
 'Wiztest.mid',
 'Woodmandrums.mid',
 'nes_po_sm03.mid',
 'Action52CityofDoom.mid',
 '8_Eyes.mid',
 'da_smb3-underwater.mid',
 '4end.mid',
 'BlasterMaster.mid',
 'MCkids-level1.mid',
 'tsb10.mid',
 'z2title.mid',
 'Golgo_13_Mafat_Conspiracy-Credits.mid',
 'normbatl.mid',
 'ptomato.mid']

In [13]:
from music21 import converter, instrument, note, chord

In [14]:
music21_objects = []

for file in random_sample:
    music21_objects.append(converter.parse(f'midi_files/{file}'))

In [15]:
sampler_parts = []

for object in music21_objects:
    score = instrument.partitionByInstrument(object)
    parts = score.parts
    for part in parts:
        if part.getInstrument().instrumentName == 'Sampler':
            sampler_parts.append(part.notes)

In [16]:
notes = []

for sampler_part in sampler_parts:
    for element in sampler_part.recurse():
        if isinstance(element, note.Note):
            notes.append(str(element.pitch) + " " +  str(element.quarterLength))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
        elif isinstance(element, note.Rest):
            notes.append(str(element.name)  + " " + str(element.quarterLength))

In [17]:
len(notes)

6555

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import utils

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / n_vocab

    network_output = utils.to_categorical(network_output)

    return (network_input, network_output)

In [19]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
import os
from tensorflow.keras.callbacks import ModelCheckpoint

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = keras.Sequential([
#         keras.layers.Embedding(input_dim=len(network_input), output_dim=n_vocab),
        LSTM(256,
             input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
             return_sequences=True
            ),
        LSTM(256, return_sequences=True),
        Flatten(),
        Dense(n_vocab, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="training_1/cp.ckpt",
                                                     save_weights_only=True,
                                                     verbose=1)

    model.fit(network_input, network_output, epochs=25, batch_size=64, callbacks=[cp_callback])

In [20]:
def train_network(notes, n_vocab):
    """ Train a Neural Network to generate music """
    network_input, network_output = prepare_sequences(notes, n_vocab)
    global model
    model = create_network(network_input, n_vocab)
    train(model, network_input, network_output)

In [21]:
n_vocab = len(set(notes))
train_network(notes, n_vocab)

Epoch 1/25
101/101 [==============================] - 41s 387ms/step - loss: 5.6113

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/25
101/101 [==============================] - 39s 385ms/step - loss: 4.9656

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/25
101/101 [==============================] - 38s 377ms/step - loss: 4.5136

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/25
101/101 [==============================] - 39s 383ms/step - loss: 3.9912

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/25
101/101 [==============================] - 38s 380ms/step - loss: 3.4155

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/25
101/101 [==============================] - 38s 381ms/step - loss: 2.7995

Epoch 00006: saving model to training_1/cp.ckpt
Epoch 7/25
101/101 [==============================] - 38s 378ms/step - loss: 2.3228

Epoch 00007: saving model to training_1/cp.ckpt
Epoch 8/25
101/101 [==============================] - 42s 419ms/step 

In [22]:

def prepare_sequences_output(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [23]:
from music21 import stream

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        pattern = pattern.split()
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif('rest' in pattern):
            new_rest = note.Rest(pattern)
            new_rest.offset = offset
            new_rest.storedInstrument = instrument.Piano() #???
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += convert_to_float(duration)

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')
 
# From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [27]:
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))

network_input, normalized_input = prepare_sequences_output(notes, pitchnames, n_vocab)
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
create_midi(prediction_output)